In [1]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import pandas as pd
import numpy as np
import numexpr
import os
import torch
import pickle

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

from botorch.acquisition.multi_objective.monte_carlo import (
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.sampling import sample_simplex

import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
print(tkwargs)

E:\conda\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dtype': torch.float64, 'device': device(type='cpu')}


In [2]:
# ini_constraints = ['x1 + x2 + x3 - ']

# def check_constraints(x):
#     return (x.sum(dim=-1) - 200).view(-1,1)

# def generate_initial_data(n, bounds):
#     # generate training data
#     train_x = draw_sobol_samples(bounds=bounds, n=n, q=1).squeeze(1)
#     train_con = check_constraints(train_x)
    
#     return train_x, train_con

def initialize_model(train_x, train_obj, bounds):
    # define models for objective and constraint
    train_x = normalize(train_x, bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i:i+1]
        # train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            SingleTaskGP(train_x, train_y, outcome_transform=Standardize(m=1)) 
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model


def optimize_qnehvi(model, bounds, train_x, train_obj, sampler, inequality_constraints=None):
    """Optimizes the qNEHVI acquisition function, and returns a new candidate and observation."""
    standard_bounds = torch.zeros(2, train_x.shape[-1], **tkwargs)
    standard_bounds[1] = 1
    train_x = normalize(train_x, bounds)
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=True,
        # define an objective that specifies which outcomes are the objectives
        objective=IdentityMCMultiOutputObjective(outcomes=[i for i in range(train_obj.shape[-1])]),
        # specify that the constraint is on the last outcome
        # constraints=[lambda Z: Z[..., -1]],
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 400},
        sequential=True,
        inequality_constraints=inequality_constraints
    )
    
    return candidates

def pre_by_model(models, x, bounds):
    test_x = normalize(x, bounds)
    results = None
    for i in range(len(models)):
        a = models[i].posterior(test_x).mean.detach().cpu().numpy()
        b = models[i].posterior(test_x).variance.detach().cpu().numpy()
        b=np.sqrt(b)
        c = np.concatenate([a,b], axis=-1)
        if results is None:
            results = c
        else:
            results = np.concatenate([results, c], axis=0)
    return results

In [3]:
def get_new_df(new_x, model_qnehvi, bounds):
    result = pre_by_model(model_qnehvi.models, new_x, bounds)
    new_x = new_x.round().detach().cpu().numpy()
    select_df1 = pd.DataFrame({'Temperature1/C': new_x[:,0],
                               'Time1/min': new_x[:,1],
                               'Temperature2/C': new_x[:,2],
                               'Time2/min': new_x[:,3],
                               'Temperature3/C': new_x[:,4],
                               'Time3/min': new_x[:,5]})
    for i in range(1, 4):
        a = np.array(select_df1['Time%i/min'%i])//60
        b = np.array(select_df1['Time%i/min'%i]) - a *60
        time = ['%sh%smin'%(int(a[i]), int(b[i])) for i in range(BATCH_SIZE)]
        select_df1['Time%i/h'%i] = time

    select_df1[['pre_strength', 'pre_module', 'pre_elongation', 'pre_tg']] =  result[:,0].reshape(-1, BATCH_SIZE).T
    select_df1[['pre_strength_bar', 'pre_module_bar', 'pre_elongation_bar', 'pre_tg_bar']] =  result[:,1].reshape(-1, BATCH_SIZE).T
    
    select_df1 = select_df1[[
        'Temperature1/C', 'Time1/min', 'Temperature2/C', 'Time2/min',
        'Temperature3/C', 'Time3/min', 'Time1/h', 'Time2/h', 'Time3/h',
        'pre_strength', 'pre_strength_bar', 'pre_module','pre_module_bar',
        'pre_elongation', 'pre_elongation_bar', 'pre_tg', 'pre_tg_bar'        
    ]]
    return select_df1

def get_ehvi(model_qnehvi, ref_point, train_x, train_obj, can):
    qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model_qnehvi,
            ref_point= ref_point, # current_state['ref_point'].tolist(),  # use known reference point
            X_baseline=train_x,
            sampler=qnehvi_sampler,
            prune_baseline=True,
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[i for i in range(train_obj.shape[-1])]),
            # specify that the constraint is on the last outcome
            # constraints=[lambda Z: Z[..., -1]],
        )
    result = [acq_func(can[i].view(1,-1)).detach().item() for i in range(can.shape[0])]
    return result

In [4]:
x_dim = 6
bounds = torch.tensor([[95., 120., 130., 120., 170., 120.],
                       [125., 300., 160., 300., 190., 300.]] , **tkwargs)

BATCH_SIZE = 4
NUM_RESTARTS = 10 
RAW_SAMPLES = 2048 
N_BATCH = 20 
MC_SAMPLES = 1024 

In [5]:
df = pd.read_csv('../data/initial.csv')
df = df.sample(frac=1.0)

train_x = np.array(df[['Temperature1/C', 'Time1/min', 'Temperature2/C','Time2/min', 'Temperature3/C', 'Time3/min']])
train_x = torch.tensor(train_x, **tkwargs)
train_obj = torch.tensor(np.array(df[['tensile strength/MPa','Young\'s modulus/Mpa', 'elongation_break/%', 'Tg']]), **tkwargs)

mll_qnehvi, model_qnehvi = initialize_model(train_x, train_obj, bounds)
fit_gpytorch_mll(mll_qnehvi)

# Group A

In [ ]:
ref_point = train_obj.mean(axis=0)

hv = Hypervolume(ref_point=ref_point)

if train_obj.shape[0] > 0:
    pareto_mask = is_non_dominated(train_obj)
    pareto_y = train_obj[pareto_mask]
    # compute hypervolume
    volume = hv.compute(pareto_y)
else:
    volume = 0.0

hvs = []
hvs.append(volume)
print(hvs)

In [ ]:
qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
candidates = optimize_qnehvi(model_qnehvi, bounds, train_x, train_obj, qnehvi_sampler)

new_x =  unnormalize(candidates.detach(), bounds=bounds)
new_x = new_x.round()
new_df = get_new_df(new_x, model_qnehvi, bounds)
ehvi = get_ehvi(model_qnehvi, ref_point.tolist(), train_x, train_obj, candidates)
new_df['ehvi'] = [ehvi[i] for i in range(len(ehvi))]
# new_df.to_csv('select.csv')

# Group B

In [10]:
ref_point = train_obj.mean(axis=0)
ref_point = ref_point + train_obj.std(axis=0) * torch.tensor([-1, 3.14, -1, -1], **tkwargs)
hv = Hypervolume(ref_point=ref_point)

if train_obj.shape[0] > 0:
    pareto_mask = is_non_dominated(train_obj)
    pareto_y = train_obj[pareto_mask]
    # compute hypervolume
    volume = hv.compute(pareto_y)
else:
    volume = 0.0

hvs = []
hvs.append(volume)
print(hvs)

[0.0]


In [ ]:
qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
candidates = optimize_qnehvi(model_qnehvi, bounds, train_x, train_obj, qnehvi_sampler)

new_x =  unnormalize(candidates.detach(), bounds=bounds)
new_x = new_x.round()
new_df = get_new_df(new_x, model_qnehvi, bounds)
ehvi = get_ehvi(model_qnehvi, ref_point.tolist(), train_x, train_obj, candidates)
new_df['ehvi'] = [ehvi[i] for i in range(len(ehvi))]
# new_df.to_csv('select.csv')

# Reduce dimension

In [20]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
import pickle

In [21]:
new_df = pd.read_csv('initial.csv')
map_bounds = np.array([[95., 120., 130., 120., 170., 120.],
                       [125., 300., 160., 300., 190., 300.]])

with open('umaper.pt', 'rb') as f:
    reducer = pickle.load(f)
    

train_x = np.array(new_df[['Temperature1/C', 'Time1/min', 'Temperature2/C','Time2/min', 'Temperature3/C', 'Time3/min']])
train_x = (train_x-map_bounds[0,:])/(map_bounds[1,:]-map_bounds[0,:])
Y = reducer.transform(train_x)
new_df[['dim1','dim2']] = Y
new_df.to_csv('initial_umap.csv', index=False)